# Cellpose segmentation

Notebook showing how to segment cells using Cellpose.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
from skimage.io import imsave
import tensorflow as tf
import cellpose
from cellpose import models
from cellpose import utils
from tifffile import imread, imsave

sys.path.append('../libraries')
import input_functions as inp_f

In [2]:
info_file_path = r'Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\livecell_B03\full\data\info_B03.txt'

In [3]:
#Add stuff to do GPU segmentation

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
#Is cellpose using the GPU, reports True or False
cellpose.core.use_gpu(gpu_number=0, use_torch=True)

True

In [5]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:36:24_Pacific_Standard_Time_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0
Tue May  7 19:29:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 538.18                 Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro P620                  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P0    

In [6]:
# load cellpose model - set gpu to False if your machine does not support CUDA based GPU segmentation
model = models.Cellpose(gpu=True, model_type='cyto')

In [7]:
# read the file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

# get info about the channels
channel_list = inp_f.read_channels(info_lines)

In [8]:
file_name = [x['file_name'] for x in channel_list if x['tracking']][0]
c = [x['channel_in_file'] for x in channel_list if x['tracking']][0]
im_path = os.path.join(exp_dir, 'data', file_name)

# Function to read a single frame from a TIFF file
def open_frame(movie_path, frame_index):
    frame = imread(movie_path, key=frame_index)
    return frame

# check how many timepoints are there in the file
frames_num = 188

print(f'Total frame number: {frames_num}')

# Loop for segmentation
for i in range(frames_num):
    # Define save directory and file name
    save_dir = os.path.join(exp_dir, 'segmentation')
    save_file = f"{file_name[:-4]}_{str(i).zfill(3)}_label.png"
    save_path = os.path.join(save_dir, save_file)

    # Check if the file already exists
    if os.path.exists(save_path):
        print(f"Frame {i+1}/{frames_num} already segmented. Skipping...")
        continue

    # Get an image frame
    print("Building im_frame")
    im_frame = open_frame(im_path, i)

    # Print progress
    print(f"Segmenting frame {i+1}/{frames_num}")

    # Segment the right plane
    labels, _, _, _ = model.eval(im_frame, diameter=30, channels=[0, 0])
    print("Segmenting correct plane")

    # Save segmentation
    imsave(save_path, labels.astype('uint16'))
    print(f"Frame {i+1}/{frames_num} saved")

print("Segmentation completed.")

Total frame number: 188
Frame 1/188 already segmented. Skipping...
Frame 2/188 already segmented. Skipping...
Frame 3/188 already segmented. Skipping...
Frame 4/188 already segmented. Skipping...
Frame 5/188 already segmented. Skipping...
Frame 6/188 already segmented. Skipping...
Frame 7/188 already segmented. Skipping...
Frame 8/188 already segmented. Skipping...
Frame 9/188 already segmented. Skipping...
Frame 10/188 already segmented. Skipping...
Frame 11/188 already segmented. Skipping...
Frame 12/188 already segmented. Skipping...
Frame 13/188 already segmented. Skipping...
Frame 14/188 already segmented. Skipping...
Frame 15/188 already segmented. Skipping...
Frame 16/188 already segmented. Skipping...
Frame 17/188 already segmented. Skipping...
Frame 18/188 already segmented. Skipping...
Frame 19/188 already segmented. Skipping...
Frame 20/188 already segmented. Skipping...
Frame 21/188 already segmented. Skipping...
Frame 22/188 already segmented. Skipping...
Frame 23/188 alre